In [2]:
import pandas as pd
import numpy as np
from statsmodels.stats.inter_rater import fleiss_kappa

# Raw data from the table
data = {
    "Plant name": [
        "Aloe ferox", "African ginger", "Wild rosemary", "Devil's claw",
        "African wormwood tree", "Pepperbark tree", "Pineapple flower",
        "Spekboom", "False horsewood", "Sand raisin", "Mountain nettle",
        "Acacia", "River karee", "Kudu lily", "Waterberg raisin",
        "Sweet wild garlic", "Cyrtanthus sanguineus", "Ruttya fruticosa",
        "Sesamum trilobum", "Aloe hahnii"
    ],
    "ChatGPT": [
        "Medicinal", "Medicinal", "No results", "Medicinal", "Medicinal",
        "Not Accurate", "Medicinal", "Not accurate", "Medicinal, Poisonous",
        "No results", "Not accurate", "Medicinal, Poisonous", "Medicinal",
        "Medicinal", "Not accurate", "No results", "Not accurate", "No results",
        "No results", "No results"
    ],
    "Gemini": [
        "Medicinal", "Medicinal", "No results", "Medicinal", "Medicinal",
        "Not accurate", "Medicinal", "Not accurate", "No results",
        "Not accurate", "Not accurate", "Poisonous", "Umhlakaza (isiZulu)",
        "Not accurate", "Not accurate", "No results", "Not accurate",
        "Medicinal", "No results", "Medicinal"
    ],
    "Mistral AI": [
        "Edible", "Medicinal", "No results", "Medicinal", "Medicinal",
        "Not accurate", "Medicinal", "Not accurate", "Not accurate",
        "Edible, Medicinal", "Not accurate", "Poisonous", "Not accurate",
        "Medicinal", "Not accurate", "Not accurate", "Edible", "Not accurate",
        "Edible", "Not accurate"
    ]
}

df = pd.DataFrame(data)

# Clean and standardize labels (lowercase, trim, split commas)
def clean_labels(column):
    return (
        column.str.lower()
        .str.strip()
        .str.replace(r'\s*,\s*', ',', regex=True)
    )

df["ChatGPT"] = clean_labels(df["ChatGPT"])
df["Gemini"] = clean_labels(df["Gemini"])
df["Mistral AI"] = clean_labels(df["Mistral AI"])

# Include "no results" as a category
all_labels = set()
for col in ["ChatGPT", "Gemini", "Mistral AI"]:
    all_labels.update(df[col].str.split(',').explode().unique())
all_labels = sorted(all_labels)

# Function to compute Fleiss' Kappa per plant
def fleiss_kappa_per_plant(row):
    ratings = [row["ChatGPT"], row["Gemini"], row["Mistral AI"]]
    expanded = [label.split(',') for label in ratings]

    # Count occurrences of all labels (including "no results")
    counts = {label: 0 for label in all_labels}
    for sublist in expanded:
        for label in sublist:
            counts[label] += 1

    matrix_row = np.array([counts[label] for label in all_labels])

    # Edge case: All raters agree on one category (even "no results")
    non_zero = [c for c in matrix_row if c > 0]
    if len(non_zero) == 1 and sum(matrix_row) == 3:
        return 1.0  # Perfect agreement

    # Edge case: All raters have "no results"
    if all(label == "no results" for label in ratings):
        return 1.0  # Perfect agreement (all chose "no results")

    # Calculate Fleiss' Kappa
    try:
        return fleiss_kappa(matrix_row.reshape(1, -1))
    except:
        # Fallback for invalid matrices (e.g., no variance)
        return 0.0  # Treat as random agreement

df["Fleiss Kappa"] = df.apply(fleiss_kappa_per_plant, axis=1)

# Show results
print(df[["Plant name", "Fleiss Kappa"]])

               Plant name  Fleiss Kappa
0              Aloe ferox     -0.500000
1          African ginger      1.000000
2           Wild rosemary      1.000000
3            Devil's claw      1.000000
4   African wormwood tree      1.000000
5         Pepperbark tree      1.000000
6        Pineapple flower      1.000000
7                Spekboom      1.000000
8         False horsewood     -0.333333
9             Sand raisin     -0.333333
10        Mountain nettle      1.000000
11                 Acacia     -0.333333
12            River karee     -0.500000
13              Kudu lily     -0.500000
14       Waterberg raisin      1.000000
15      Sweet wild garlic     -0.500000
16  Cyrtanthus sanguineus     -0.500000
17       Ruttya fruticosa     -0.500000
18       Sesamum trilobum     -0.500000
19            Aloe hahnii     -0.500000
